In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [50]:
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
sys.path.append('..')
from shared.models import *
from shared.datasets import *

def train(model, train_loader, criterion, device, optimizer):
    model.train()
    train_loss = 0
    for step, (data_inputs, data_labels) in enumerate(train_loader):
        inputs, labels = data_inputs.to(device), data_labels.to(device) # Convert Tensors to appropriate device
        optimizer.zero_grad() 
        pred = model(inputs)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() # Running training loss
        
    return train_loss/step

def test(model, test_loader, criterion, device, n_way):
    # An F1 Score of 0 indicates that it is invalid
    model.eval()
    true_positive = list(0. for i in range(n_way)) # Number of correctly predicted samples per class
    total_truth = list(0. for i in range(n_way)) # Number of ground truths per class
    predicted_positive = list(0. for i in range(n_way)) # Number of predicted samples per class
    precision = list(0. for i in range(n_way))
    recall = list(0. for i in range(n_way))
    class_f1 = list(0. for i in range(n_way))
    val_loss = 0
    correct_total = 0 # Total correctly predicted samples
    total = 0 # Total samples
    f1_flag = 0 # Flag for invalid F1 score
    with torch.no_grad():
        for step, (data_inputs, data_labels) in enumerate(test_loader):
            inputs, labels = data_inputs.to(device), data_labels.to(device)
            pred = model(inputs)
            loss = criterion(pred, labels)
            val_loss += loss.item() # Running validation loss
            _, predicted = torch.max(pred, 1)
            correct = (predicted == labels).squeeze() # Samples that are correctly predicted
            correct_total += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(predicted)):
                label = labels[i]
                true_positive[label] += correct[i].item() 
                total_truth[label] += 1
                predicted_positive[predicted[i].item()] += 1 # True Positive + False Positive
                
        # Find class accuracy, precision and recall
        for j in range(n_way):
            if (predicted_positive[j] != 0 and true_positive[j] != 0): # Check if F1 score is valid
                precision[j] = true_positive[j] / predicted_positive[j]
                recall[j] = true_positive[j] / total_truth[j] # Recall is the same as per class accuracy
                class_f1[j] = 2 * precision[j] * recall[j] / (precision[j] + recall[j])
            else:
                f1_flag = 1
                
        # Find Accuracy, Macro Accuracy and Macro F1 Score
        macro_acc_sum = 0
        f1_sum = 0
        for k in range(n_way):
            macro_acc_sum += recall[k]
            if f1_flag == 0: # Check for invalid f1 score
                f1_sum += class_f1[k]
                
        accuracy = correct_total/total
        macro_accuracy = macro_acc_sum/n_way 
        f1_score = f1_sum/n_way
        
    return val_loss/step, accuracy, macro_accuracy, f1_score, class_f1

if __name__ == '__main__':
    # Set Training Parameters
    num_epochs = 5
    num_workers = 8
    bs = 64
    n_way = 3
    path_results = '../../results/basic.csv' # Full path to save the CSV results
    path_models = '../../models/basic' # Folder path to save the trained models to
    save_models = True # Whether to save the trained models (Occurs every epoch)
    
    torch.cuda.set_device(0)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = BaselineNet(n_way).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Load in data
    train_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path='./splits.csv', mode='novel_train', resize=224)
    test_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path='./splits.csv', mode='novel_validate', resize=224)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
    
    # Create Dataframe to export results to CSV
    df_results = pd.DataFrame(columns=['Epoch', 'Training Loss', 'Validation Loss','Accuracy','Macro Accuracy',
                                       'Macro-F1 Score'] + [str(x)+' F1' for x in range(n_way)])
    
    # Training Loop
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, device, optimizer)
        val_loss, acc, m_acc, macro_f1, class_f1 = test(model, test_loader, criterion, device, n_way)
        
        if (save_models):
            torch.save(model.state_dict(), os.path.join(path_models, f'basic_{epoch+1}.pth')) # Save the model
            
        # Append and report results
        df_results.loc[epoch] = [epoch+1, train_loss, val_loss, acc, m_acc, macro_f1] + class_f1
        print(f'[{epoch+1}] t_loss: {train_loss:.5f} v_loss: {val_loss:.5f} val_acc: {acc:.5f} val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')
        
    df_results.to_csv(path_results, index=False) # Export results to a CSV file

[1] t_loss: 1.52318 v_loss: 1.17755 val_acc: 0.33333 val_m_acc: 0.33333 f1: 0.00000
[2] t_loss: 1.24444 v_loss: 1.17966 val_acc: 0.34333 val_m_acc: 0.34333 f1: 0.00000
[3] t_loss: 1.09972 v_loss: 1.18735 val_acc: 0.33778 val_m_acc: 0.33778 f1: 0.00000
[4] t_loss: 0.98139 v_loss: 1.18617 val_acc: 0.42222 val_m_acc: 0.42222 f1: 0.00000
[5] t_loss: 0.87207 v_loss: 1.20205 val_acc: 0.37889 val_m_acc: 0.37889 f1: 0.00000


In [46]:
os.path.join('test','bruh')

'test/bruh'

In [37]:
model.state_dict()

OrderedDict([('block1.0.weight',
              tensor([[[[ 2.8213e-01, -7.1140e-02, -5.7429e-02],
                        [ 2.2888e-01, -2.8346e-01, -3.0764e-01],
                        [ 2.2086e-02,  7.1106e-03,  5.3982e-02]]],
              
              
                      [[[ 7.4638e-02, -1.2696e-01, -2.4667e-01],
                        [ 2.6598e-01, -1.8229e-01,  3.1379e-01],
                        [ 1.3060e-02,  5.4480e-02,  2.7078e-01]]],
              
              
                      [[[ 3.0275e-01,  1.2773e-01, -1.7708e-01],
                        [ 1.7360e-01, -2.7249e-01,  4.2134e-02],
                        [ 1.3960e-01,  1.4359e-01, -2.6540e-01]]],
              
              
                      [[[ 2.3247e-01, -2.1367e-01, -1.0137e-01],
                        [ 1.1483e-01,  2.2437e-01, -7.0463e-02],
                        [-2.2381e-01,  3.2220e-01,  2.9854e-01]]],
              
              
                      [[[-9.8231e-02,  8.4951e-02, -2.8549e

In [39]:
torch.save(model.state_dict(), '../results/test.pth')

In [44]:
path_save = './../models/basic/' + f'{epoch}'

In [28]:
correct = (predicted == labels).squeeze()
for i in range(len(predicted)):
    label = labels[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

In [45]:
path_save

'./../models/basic/19'

In [37]:
total = 0
for j in range(n_way):
    total += class_correct[j] / class_total[j]
accuracy = total/n_way

0.0
0.6666666666666666
0.0
0.3333333333333333
0.0
0.2222222222222222
0.14285714285714285
0.0
0.0
0.5
0.1865079365079365


In [35]:
test = 0
test += 1.5
test

1.5

In [21]:
c

tensor([False,  True, False, False, False, False, False, False, False, False,
        False,  True,  True, False, False, False, False, False, False,  True,
        False,  True, False, False,  True, False, False, False,  True, False,
        False,  True, False, False, False], device='cuda:0')